In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
data_path = Path('./raw_data')
results_path = Path('./results')

In [119]:
# need: 
# A_p, A_g, generators, lines, load, gas_supply, gas_demand, pipelines, nodes_p, nodes_g

time_index = 2200

regions = pd.read_csv(data_path / "regions.csv").set_index("model_region")
gens_full = pd.read_csv(data_path / "Generators_data.csv")
region_map = gens_full[["region","Zone"]].drop_duplicates().set_index("region").Zone.to_dict()
gens_full["Region_Tech"] = gens_full["region"] + "_" + gens_full["technology"]
gens_full = gens_full.loc[gens_full.FLEX == 0,:] # remove flexible load resources
gens_full = gens_full.loc[gens_full.STOR == 0,:] # remove storage resources

# add variability
variability = pd.read_csv(data_path / "Generators_variability.csv").set_index("Time_Index").loc[time_index,:].to_dict()
gens_full["variability"] = gens_full["Resource"].map(variability)
gens_full["max_cap_mw"] = gens_full["Existing_Cap_MW"] * gens_full["variability"]

gens_summary = gens_full.pivot_table(
    index=["region","Zone","Region_Tech"],
    values=["max_cap_mw","Heat_Rate_MMBTU_per_MWh","Fuel"],
    aggfunc = {
        "max_cap_mw":np.sum,
        "Heat_Rate_MMBTU_per_MWh":np.mean,
        "Fuel": min,
    }
).reset_index().set_index("Region_Tech")

# get fuel info
fuel_prices = pd.read_csv(data_path / "Fuels_data.csv").set_index("Time_Index").loc[time_index,:].to_dict()
gens_summary["fuel_price_dollars_per_mmbtu"] = gens_summary.Fuel.map(fuel_prices)
gens_summary["fuel_cost_dollars_per_mwh"] = gens_summary["fuel_price_dollars_per_mmbtu"] * gens_summary["Heat_Rate_MMBTU_per_MWh"]

# add remaining columns
gens_summary["efficiency_gj_in_per_mwh_out"] = gens_summary["Heat_Rate_MMBTU_per_MWh"] * 1.055056
gens_summary["min_cap_mw"] = 0
gens_summary["is_gas"] = gens_summary.Fuel.str.contains("gas")
gens_summary["node_p"] = gens_summary["Zone"] - 1
gens_summary["node_g"] = gens_summary.node_p
gens_summary.index.name="name"
generators = gens_summary[["node_p","node_g","is_gas","min_cap_mw","max_cap_mw","fuel_cost_dollars_per_mwh","efficiency_gj_in_per_mwh_out"]]

# get load

load = pd.read_csv(data_path / "Load_data.csv")
load = load[load.columns[load.columns.str.contains("Load_MW")]]
load.columns = [x.split("_")[2][1:] for x in load.columns]
load = load.loc[time_index]
load.index.name = "node"
load.index = load.index.astype(int)
load.index = load.index - 1
load.name = "load_mw"
load = load.reset_index()

In [120]:
# network information

network = pd.read_csv(data_path / "Network.csv")

network_summary = network[network.columns[network.columns.str.contains("z")]].drop(columns="Network_zones")
network_summary.columns = [x[1:] for x in network_summary.columns]
lines = network_summary.apply(lambda row: {
        "from_node":row.index[row==1][0],
        "to_node":row.index[row==-1][0],
    },axis=1,result_type="expand")
lines["reactance_pu"] = 0.3
lines["capacity_mw"] = network["Line_Max_Flow_MW"]

network_summary.columns.name = "nodes"
network_summary.columns = network_summary.columns.astype(int)
network_summary.index.name = "edges"
network_summary.columns = network_summary.columns - 1

A_p = network_summary.transpose()

A_g = A_p.copy(deep=True)

gas_supply = load.copy(deep=True)
gas_supply = gas_supply.rename(columns={"load_mw":"supply_gj","node":"node_g"})
gas_supply["supply_gj"] = 1e9

gas_demand = gas_supply.copy(deep=True)
gas_demand = gas_demand.rename(columns={"supply_gj":"demand_gj"})
gas_demand["demand_gj"] = 1e3

pipelines = lines.copy(deep=True)
pipelines = pipelines.rename(columns={"reactance_pu":"capacity_gj","capacity_mw":"cost"})
pipelines["capacity_gj"] = 1e9
pipelines["cost"] = 5

In [121]:
node_locations = pd.read_csv(data_path / "regions.csv")
node_locations["node"] = node_locations.model_region.map(region_map)
node_locations["node"] = node_locations.node - 1
nodes_p = node_locations[["node","lat","lon"]].set_index("node")
nodes_g = nodes_p.copy(deep=True)

In [128]:
# A_p, A_g, generators, lines, load, gas_supply, gas_demand, pipelines, nodes_p, nodes_g

A_p.to_csv(results_path / "A_p.csv")
A_g.to_csv(results_path / "A_g.csv")
generators.to_csv(results_path / "generators.csv")
lines.to_csv(results_path / "lines.csv",index=False)
load.to_csv(results_path / "load.csv",index=False)
gas_supply.to_csv(results_path / "gas_supply.csv",index=False)
gas_demand.to_csv(results_path / "gas_demand.csv",index=False)
pipelines.to_csv(results_path / "pipelines.csv",index=False)
nodes_p.to_csv(results_path / "nodes_p.csv")
nodes_g.to_csv(results_path / "nodes_g.csv")

In [126]:
lines

,from_node,to_node,reactance_pu,capacity_mw
edges,,,,
0,1,2,0.3,100.0
1,1,3,0.3,5920.0
2,1,10,0.3,3350.0
3,1,17,0.3,3750.0
4,1,24,0.3,5385.0
5,2,3,0.3,3675.0
6,2,10,0.3,3675.0
7,3,10,0.3,2858.0
8,3,24,0.3,4354.0
